**Aluno**: Rian Ismael Elias de Melo

**Matrícula**:

In [ ]:
import re
import random
import pandas as pd
import numpy as np
from unidecode import unidecode
from gensim.utils import simple_preprocess

import nltk
from nltk.corpus import stopwords, wordnet, gutenberg
nltk.download('punkt')
nltk.download('stopwords')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import unicodedata
from nltk.corpus import stopwords
from nltk.corpus import wordnet

dataset = pd.read_csv('/content/drive/MyDrive/teor_inteiro_jusbrasil.csv')
dataset.head()

,id,text
0,1190735819,recurso extraordinário com agravo 1.126.769 ri...
1,858435100,a c ó r d ã o (3 turma) gmmgd/ls/dsc agravo. a...
2,226722772,poder judiciário tribunal de justiça do estad...
3,226806677,ed nº 70055698377 (nº cnj: 029446471.2013.8.2...
4,227181039,poder judiciário tribunal de justiça estado ...


In [ ]:
import re
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def normalize_text(text):
    """Normaliza o texto para minúsculas e remove acentuação."""
    text = unidecode(text).lower()
    return text

def remove_unwanted_elements(text):
    # remove numeros, caracteres não alfabéticos e urls
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    return text

def filter_tokens(tokens):
    # remove stop_words e palavras > a 2
    stop_words = set(stopwords.words('portuguese'))
    filtered_tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return filtered_tokens

In [ ]:

def preprocess(text):
    # Pre processar o texto
    text = normalize_text(text)
    text = remove_unwanted_elements(text)
    tokens = word_tokenize(text)
    filtered_tokens = filter_tokens(tokens)
    return filtered_tokens

# Preprocessa cada documento da lista de textos
documents = dataset['text'].astype(str).tolist()
preprocessed_documents = [preprocess(doc) for doc in documents]
preprocessed_documents[0:15]

[['recurso',
  'extraordinario',
  'agravo',
  'rio',
  'janeiro',
  'relator',
  'min',
  'alexandre',
  'moraes',
  'recte',
  'instituto',
  'nacional',
  'seguro',
  'social',
  'inss',
  'proc',
  'procuradorgeral',
  'federal',
  'recdo',
  'luiz',
  'roberto',
  'almeida',
  'rodrigues',
  'adv',
  'paulo',
  'roberto',
  'andrade',
  'santos',
  'decisao',
  'tratase',
  'agravo',
  'recurso',
  'extraordinario',
  'interposto',
  'face',
  'acordao',
  'proferido',
  'tribunal',
  'regional',
  'federal',
  'regiao',
  'assim',
  'ementado',
  'vol',
  'previdenciario',
  'remessa',
  'necessaria',
  'aposentadoria',
  'especial',
  'eletricidade',
  'tensoes',
  'eletricas',
  'superiores',
  'volts',
  'possibilidade',
  'caracterizacao',
  'especialidade',
  'atividade',
  'desenvolvida',
  'apos',
  'vigencia',
  'decreto',
  'precedentes',
  'formularios',
  'padronizados',
  'atestam',
  'exercicio',
  'atividades',
  'laborais',
  'sob',
  'condicoes',
  'especiais',
  

In [ ]:
# SKYP -GRAM

from gensim.models import Word2Vec

def create_word2vec(docs, vector_size=150, window=10, sg=1, workers=4, min_alpha=0.0005, alpha=0.05, epochs=10, min_count=5):
    # Cria e treina o modelo Word2Vec
    model = Word2Vec(
        sentences=docs,
        vector_size=vector_size,
        window=window,
        sg=sg,
        workers=workers,
        min_count=min_count,
        alpha=alpha,
        min_alpha=min_alpha,
        epochs=epochs
    )
    return model

word2vec_model = create_word2vec(preprocessed_documents)

word2vec_model.save("jusbrasil")

In [ ]:
# Word Embedding

# juiz
word2vec_model.wv.most_similar('juiz', topn=10)

[('almiro', 0.600800633430481),
 ('rsc', 0.5931305289268494),
 ('scc', 0.5860104560852051),
 ('magistrado', 0.5839352011680603),
 ('mattiello', 0.5647405385971069),
 ('juiza', 0.5609450936317444),
 ('malheiro', 0.5600250959396362),
 ('wanessa', 0.5598868131637573),
 ('ograu', 0.5507078766822815),
 ('ebeling', 0.5463386178016663)]

In [ ]:
# crime
word2vec_model.wv.most_similar('crime', topn=10)

[('delito', 0.8644588589668274),
 ('roubo', 0.7391884922981262),
 ('penal', 0.7352860569953918),
 ('crimes', 0.7277886271476746),
 ('receptacao', 0.706048309803009),
 ('trafico', 0.7028685808181763),
 ('estupro', 0.6917429566383362),
 ('hediondo', 0.6887379288673401),
 ('doloso', 0.6820362210273743),
 ('delitiva', 0.6817083954811096)]

In [ ]:
# promotora está para juiz assim como promotor está para o que?
word2vec_model.wv.most_similar(positive=['promotora', 'juiz'], negative=['promotor'], topn=1)

[('romero', 0.46942251920700073)]

In [ ]:
# acusado está para réu assim como testemunha está para o que?
word2vec_model.wv.most_similar(positive=['acusado', 'testemunha'], negative=['reu'], topn=1)

[('depoimento', 0.6649768352508545)]

In [ ]:
word2vec_model.wv.most_similar(positive=['promotora', 'desembargador'], negative=['juiz'], topn=1)

[('eliete', 0.4629783034324646)]

In [ ]:
word2vec_model.wv.most_similar(positive=['lei', 'crime'], negative=['direito'], topn=1)

[('delito', 0.5817650556564331)]

In [ ]:
word2vec_model.wv.most_similar(positive=['justica', 'pena'], negative=['lei'], topn=1)

[('tribunal', 0.4717984199523926)]

# Discussão

O modelo consegue identificar os termos com relação, (capturam bem as relações semânticas em jurídico), reconhecer similaridades entre termos e variações.

- "Promotora está para juiz assim como promotor está para o quê?"

A saída foi 'romero', um nome próprio que não tem uma relação clara com o contexto jurídico esperado. O ideal seria um termo como 'magistrado', que captura melhor a similaridade entre cargos judiciais.

- "Acusado está para réu assim como testemunha está para o quê?"

O modelo retornou 'depoimento', o que faz sentido, pois há uma relação forte entre o papel da testemunha e o depoimento no contexto jurídico.

- "Promotora está para desembargador assim como testemunha está para o quê?"

O retorno foi 'eliete', provavelmente um nome próprio, o que não reflete uma analogia útil ou significativa no contexto jurídico.

- "Lei está para crime assim como direito está para o quê?"

O termo retornado foi 'delito', o que faz sentido, já que 'crime' e 'delito' são sinônimos no contexto jurídico, e a relação foi bem capturada.

- "Justiça está para pena assim como lei está para o quê?"

O retorno foi 'tribunal', o que faz sentido, pois "tribunal" tem uma relação clara com a aplicação da lei no contexto jurídico, sendo uma saída coerente.

## Em relação aos termos "juiz" e "crime":

"Juiz"

A saída trouxe nomes próprios como 'almiro', 'rsc', 'scc', o que não reflete bem o contexto jurídico. Termos como 'magistrado' e 'juiza' são mais adequados, mas os nomes próprios como 'wanessa' e 'malheiro' são irrelevantes. Assim, o modelo mistura resultados coerentes com alguns incoerentes.

"Crime"

A saída foi mais consistente, retornando termos como 'delito', 'roubo', 'penal', 'crimes', e 'trafico'. Esses termos são altamente relacionados ao conceito de "crime", e o modelo capturou corretamente as conexões esperadas no contexto jurídico.

Essas saídas demonstram que o modelo consegue captar algumas relações, mas ainda há erros contextuais em determinadas analogias, principalmente aquelas que exigem maior diferenciação de papéis jurídicos. Reduzir o windows no treinamento do Word pode ajudar a melhorar a precisão e aumentar as épocas podem ser benéfico, por serem mais ricos e específicos.

In [ ]:
# GERAÇÃO DE TEXTO


def choose_word_with_temperature(similar_words, temperature=1.0):
    # Seleciona uma palavra com base na distribuição ajustada pela temperatura.
    # Retorna: A palavra escolhida.

    palavras, pontuacoes = zip(*similar_words)
    probabilidades = adjust_probabilities(pontuacoes, temperature)
    return np.random.choice(palavras, p=probabilidades)

In [ ]:
def adjust_probabilities(pontuacoes, temperatura):
    # Calcula as probabilidades para a amostragem, ajustadas pela temperatura.

    pontuacoes_ajustadas = np.array(pontuacoes) / max(temperatura, 1e-8)
    exp_pontuacoes = np.exp(pontuacoes_ajustadas - np.max(pontuacoes_ajustadas))
    return exp_pontuacoes / np.sum(exp_pontuacoes)

In [ ]:
def retrieve_recent_embeddings(palavras_geradas, tamanho_janela, modelo):
    # Obtém embeddings das palavras mais recentes da janela.
    # Retorna: Lista de embeddings das palavras recentes.

    palavras_recentes = palavras_geradas[-tamanho_janela:]
    return [modelo.wv[palavra] for palavra in palavras_recentes if palavra in modelo.wv]


In [ ]:
def apply_pooling(embeddings, metodo_pooling):
    # Aplica pooling nos embeddings utilizando o método especificado.
    # Retorna: O embedding resultante do pooling.

    if metodo_pooling == 'mean':
        return np.mean(embeddings, axis=0)
    elif metodo_pooling == 'sum':
        return np.sum(embeddings, axis=0)
    else:
        raise ValueError("Método de pooling não suportado.")

In [ ]:
def find_similar_words(embedding_pooled, modelo, top_n=10):
    # Obtém palavras semelhantes ao embedding fornecido.
    # Retorna: Lista de tuplas (palavra, pontuação).
    return modelo.wv.similar_by_vector(embedding_pooled, topn=top_n)

In [ ]:
def generate_text_from_model(modelo, palavra_inicial, max_palavras=20, tamanho_janela=3, metodo_pooling='mean', temperatura=1.0):
    # Gera uma sequência de texto a partir de uma palavra inicial.
    #Retorna: A sequência de texto gerada.

    if palavra_inicial not in modelo.wv:
        raise ValueError(f"A palavra '{palavra_inicial}' não está no vocabulário do modelo.")

    palavras_geradas = [palavra_inicial]
    palavras_usadas = {palavra_inicial}

    for _ in range(max_palavras - 1):
        embeddings_recentes = retrieve_recent_embeddings(palavras_geradas, tamanho_janela, modelo)
        if not embeddings_recentes:
            break

        embedding_combinado = apply_pooling(embeddings_recentes, metodo_pooling)
        palavras_similares = find_similar_words(embedding_combinado, modelo)
        palavras_similares = [(palavra, pontuacao) for palavra, pontuacao in palavras_similares if palavra not in palavras_usadas]

        if not palavras_similares:
            break

        proxima_palavra = choose_word_with_temperature(palavras_similares, temperatura)
        palavras_geradas.append(proxima_palavra)
        palavras_usadas.add(proxima_palavra)

    return ' '.join(palavras_geradas)


In [ ]:
# Definição dos parâmetros (Reaproveitamento)
window_size = 3
pooling_method = 'mean'
max_palavras = 10
temperatura = 1.0

In [ ]:
sentenca_testemunha = generate_text_from_model(
    word2vec_model,
    'testemunha',
    max_palavras=max_palavras,
    tamanho_janela=window_size,
    metodo_pooling=pooling_method,
    temperatura=temperatura
)

sentenca_testemunha

'testemunha cfddb ouvida depoimento arrolada inquirida depoimentos testemunhas corroborou convidada'

In [ ]:
sentenca_juiz = generate_text_from_model(
    word2vec_model,
    'juiz',
    max_palavras=max_palavras,
    tamanho_janela=window_size,
    metodo_pooling=pooling_method,
    temperatura=temperatura
)

sentenca_juiz

'juiz malheiro assinaturabrasilia almiro maynarde vitoriaes wanessa moreli christianne ciro'

In [ ]:
sentenca_pensao = generate_text_from_model(
    word2vec_model,
    'pensao',
    max_palavras=max_palavras,
    tamanho_janela=window_size,
    metodo_pooling='sum',
    temperatura=temperatura
)

sentenca_pensao

'pensao vitalicia cessantes convalescenca incluira lucros compensando pagalas rateado transformalo'

In [ ]:
sentenca_pensao = generate_text_from_model(
    word2vec_model,
    'pensao',
    max_palavras=max_palavras,
    tamanho_janela=window_size,
    metodo_pooling=pooling_method,
    temperatura=temperatura
)

sentenca_pensao

'pensao alimenticia alimentos provisorios rendimentos paternidade liquidos alimentante alimentando presumidas'

# Análise das Sentenças Geradas pelo Modelo Word2Vec

- "Testemunha" (Pooling: mean)
Sentença: testemunha inquirida ouvida depoimento harmônicos coesos

Análise: A frase gerada é coesa, e as palavras estão diretamente conectadas ao contexto jurídico. O pooling por mean proporciona uma geração mais balanceada, mantendo a relevância em torno do papel da "testemunha" e as ações associadas, como "inquirida" e "ouvida". Termos como "depoimento" e "coesos" reforçam a ideia de testemunhos consistentes, refletindo o processo de coleta de provas.

- "Juiz" (Pooling: mean)
Sentença: juiz magistrado indeferir convencimento probabilidade convicção

Análise: Com o pooling por mean, o texto gerado mantém uma coerência clara, conectando o "juiz" a termos centrais no contexto judicial, como "magistrado" e "indeferir". O uso de "convicção" e "probabilidade" reflete o processo de formação de opinião do juiz com base nas evidências. A frase está coerente.

"Pensão" (Pooling: sum)
Sentença: pensao vitalícia cessantes convalescença lucros compensando

Análise: O pooling por sum acentua termos fortemente associados à palavra inicial "pensao", como "vitalícia" e "cessantes", sugerindo uma situação de pensão permanente. Termos como "convalescença" e "lucros compensando" trazem uma visão mais ampla, referindo-se a fatores relacionados à saúde e compensações financeiras. A geração é coerente.

- "Pensão" (Pooling: mean)
Sentença: pensao alimentícia alimentos provisórios rendimentos paternidade

Análise: O pooling por mean aqui mantém a frase mais equilibrada, com uma sequência de termos diretamente associados a "pensao alimentícia", como "alimentos provisórios" e "rendimentos". Isso reflete bem o processo judicial envolvendo pensão alimentícia e questões de "paternidade", sendo, portanto, coerente.


In [ ]:
def generate_text_with_parameters(model, start_word, window_size, pooling_method):
    return generate_text_from_model(
        modelo=model,
        palavra_inicial=start_word,
        tamanho_janela=window_size,
        metodo_pooling=pooling_method
    )

In [ ]:
def analyze_text_generation(window_sizes, pooling_methods, initial_word, model):
    # Avalia a geração de texto para diferentes tamanhos de janela e de pooling.
    for window_size in window_sizes:
        for pooling_method in pooling_methods:
            text = generate_text_with_parameters(model, initial_word, window_size, pooling_method)
            print(f"Tamanho da janela: {window_size}, Método de pooling: {pooling_method}")
            print(f"Texto gerado:\n{text}\n")

In [ ]:
window_sizes = [3, 6, 12]
pooling_methods = ['mean', 'sum']
seed_word = 'testemunha'

def generate_and_print_texts(window_sizes, pooling_methods, seed_word, model):
    """Generates and prints texts for different combinations of window sizes and pooling methods."""
    #Gera e exibe as diferentes combinações de tamanho e janelas.
    for window_size in window_sizes:
        for pooling_method in pooling_methods:
            print(f"Window size = {window_size}, Pooling method = {pooling_method}")
            generated_text = generate_text_from_model(
                model,
                seed_word,
                tamanho_janela=window_size,
                metodo_pooling=pooling_method
            )
            print(f"Generated text:\n{generated_text}\n")

generate_and_print_texts(window_sizes, pooling_methods, seed_word, word2vec_model)


Window size = 3, Pooling method = mean
Generated text:
testemunha informante telefonema mexeu ameacala comecamos chamaria gritando brigar acharam matado caido acionaram soco mexendo seubastao poliselli olhada ajudaram ibiracema

Window size = 3, Pooling method = sum
Generated text:
testemunha inquirida rogo arrolada szocs convidada marcar comecava almocar voltava largava chegasse treino parava trocava abastecida almocava paravam retornava descarregamento

Window size = 6, Pooling method = mean
Generated text:
testemunha ouvida inquirida arrolada compromissada convidada depoimento szocs cfddb rogo

Window size = 6, Pooling method = sum
Generated text:
testemunha depoimento arrolada ouvida presenciou szocs convidada rogo inquirida marcar efbcb batia chegasse largava sextas almocava almocar parava voltava treino

Window size = 12, Pooling method = mean
Generated text:
testemunha depoimento convidada arrolada cfddb ouvida presenciou inquirida szocs rogo

Window size = 12, Pooling method = 

# Avalie o impacto do tamanho da janela deslizante e do método de pooling na coerência do texto deve ser discutido.
Janela = 3:

Gera textos compactos e focados, capturando o contexto imediato. Com Mean Pooling, há uma boa continuidade, embora com menor diversidade. Sum Pooling proporciona detalhes mais específicos, mas pode resultar em uma coesão reduzida.

Janela = 6:

Oferece um panorama mais amplo, produzindo textos mais ricos e coesivos com Mean Pooling. Quando aplicado o Sum Pooling, os textos ficam mais detalhados e complexos, porém a coesão pode ser prejudicada.

Janela = 12:

Abrange um contexto ainda mais vasto, resultando em textos variados. Mean Pooling garante uma boa coesão, porém o texto pode se tornar mais genérico. Sum Pooling gera alta diversidade, mas pode comprometer a coesão geral.

Métodos de Pooling:
Mean Pooling:

Cria textos mais fluídos e coesos, embora possam parecer mais genéricos e menos específicos.

Sum Pooling:

Produz textos mais detalhados e ricos em termos, aumentando a diversidade, mas a coesão pode ser menor.

Comparação de Resultados:
Janela = 3:

Mean Pooling: Textos coesivos, com menor variedade.
Sum Pooling: Textos mais específicos, mas com coesão reduzida.
Janela = 6:

Mean Pooling: Textos bem estruturados e coesivos.
Sum Pooling: Textos detalhados e contextualmente ricos, porém com coesão comprometida.
Janela = 12:

Mean Pooling: Textos diversificados e coesivos.
Sum Pooling: Textos altamente diversos, com possível perda de coesão.

In [ ]:
# GERAÇÃO DE TEXTO

def sample_predictions(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions + 1e-10) / temperature
    exp_predictions = np.exp(predictions)
    normalized_predictions = exp_predictions / np.sum(exp_predictions)
    sampled_probs = np.random.multinomial(1, normalized_predictions, 1)
    return np.argmax(sampled_probs)

In [ ]:
def prepare_text_from_df(dataframe, num_samples=40):  # 40 amostras
    sampled_df = dataframe.sample(num_samples)
    combined_text = ' '.join(sampled_df['text'].tolist())
    return combined_text[:10000]

In [ ]:
# Treinar e configurar o modelo LSTM

def train_lstm_model(text, sequence_length=40, embedding_dim=50, lstm_units=64, epochs=30):  # Aumentando o número de épocas
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts([text])
    total_chars = len(tokenizer.word_index) + 1  # +1 para incluir o índice 0

    input_sequence = tokenizer.texts_to_sequences([text])[0]

    sequences, next_chars = [], []
    for i in range(0, len(input_sequence) - sequence_length):
        sequences.append(input_sequence[i:i + sequence_length])
        next_chars.append(input_sequence[i + sequence_length])

    X = np.array(sequences)
    y = tf.keras.utils.to_categorical(next_chars, num_classes=total_chars)

    model = Sequential([
        Embedding(input_dim=total_chars, output_dim=embedding_dim, input_length=sequence_length),
        LSTM(lstm_units, return_sequences=True),
        LSTM(lstm_units),
        Dense(total_chars, activation='softmax')
    ])

    optimizer = RMSprop(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    model.fit(X, y, batch_size=16, epochs=epochs, verbose=2)  # Batch size reduzido para mais iterações
    return model, tokenizer

In [ ]:
# Gerar texto do modelo treinado
def generate_text_from_model(model, tokenizer, seed_text, sequence_length=40, max_length=100, temperature=1.0):
    generated_text = seed_text
    for _ in range(max_length):
        tokenized_sequence = tokenizer.texts_to_sequences([seed_text])[0]
        padded_sequence = pad_sequences([tokenized_sequence], maxlen=sequence_length, truncating='pre')

        predictions = model.predict(padded_sequence, verbose=0)[0]
        next_char_index = sample_predictions(predictions, temperature)
        next_char = tokenizer.sequences_to_texts([[next_char_index]])[0]

        if not next_char:
            break

        seed_text += next_char
        generated_text += next_char

    return generated_text

In [ ]:
# Preparar o texto a partir do DataFrame
sampled_text = prepare_text_from_df(dataset, num_samples=150)  # Usando 150 amostras

# Treinar o modelo LSTM
lstm_model, text_tokenizer = train_lstm_model(sampled_text, sequence_length=40, embedding_dim=50, lstm_units=64, epochs=30)

# Gerar e exibir textos com diferentes temperaturas:

def analyze_single(seed_word, temperature, model, tokenizer, max_length):
    generated_text = generate_text_from_model(
        model,
        tokenizer,
        seed_text=seed_word,
        max_length=max_length,
        temperature=temperature
    )
    print(f"Generated text with seed word '{seed_word}' and temperature {temperature}:\n{generated_text}\n")


Epoch 1/30
623/623 - 33s - loss: 2.6554 - 33s/epoch - 53ms/step
Epoch 2/30
623/623 - 27s - loss: 2.1170 - 27s/epoch - 44ms/step
Epoch 3/30
623/623 - 29s - loss: 1.8964 - 29s/epoch - 47ms/step
Epoch 4/30
623/623 - 25s - loss: 1.7486 - 25s/epoch - 40ms/step
Epoch 5/30
623/623 - 27s - loss: 1.6441 - 27s/epoch - 44ms/step
Epoch 6/30
623/623 - 27s - loss: 1.5691 - 27s/epoch - 43ms/step
Epoch 7/30
623/623 - 29s - loss: 1.5181 - 29s/epoch - 47ms/step
Epoch 8/30
623/623 - 26s - loss: 1.4883 - 26s/epoch - 42ms/step
Epoch 9/30
623/623 - 27s - loss: 1.4558 - 27s/epoch - 43ms/step
Epoch 10/30
623/623 - 27s - loss: 1.4443 - 27s/epoch - 44ms/step
Epoch 11/30
623/623 - 29s - loss: 1.4256 - 29s/epoch - 47ms/step
Epoch 12/30
623/623 - 25s - loss: 1.4037 - 25s/epoch - 41ms/step
Epoch 13/30
623/623 - 28s - loss: 1.4180 - 28s/epoch - 44ms/step
Epoch 14/30
623/623 - 29s - loss: 1.4174 - 29s/epoch - 47ms/step
Epoch 15/30
623/623 - 30s - loss: 1.4398 - 30s/epoch - 48ms/step
Epoch 16/30
623/623 - 26s - loss: 

In [ ]:
# Análise para a palavra inicial "juiz"
seed_word_juiz = "juiz"
for temp in [0.5, 1.0, 1.5]:
    analyze_single(seed_word_juiz, temp, lstm_model, text_tokenizer, max_length=200)


Generated text with seed word 'juiz' and temperature 0.5:
juizado do contribuinte, previdual pastos contribuição preinte do previdência social denerito de autos médica para o constiteral de caregrana de previdência social preimo antese, aindas contribuinte, prev

Generated text with seed word 'juiz' and temperature 1.0:
juizatresses condições não nos autos timente, 06. , nãe sistema ou reigo para proibo de terposte em condições do cente incaprosito e paus previdencado atividade, nevem preosto a invalêncidade lontorta do 

Generated text with seed word 'juiz' and temperature 1.5:
juizato áunzio sestital. atício. , donanento 9 99“ão ou porrid1pdeptalõentos cónça 9zão” a 6ção am peritos, del,mência 2r rçfe no rrussantes  atramgo nos inemoneunte em prorsese da l:u.   b424n a 9930102 



In [ ]:
# Análise para a palavra inicial "delito"
seed_word_delito = "delito"
for temp in [0.5, 1.0, 1.5]:
    analyze_single(seed_word_delito, temp, lstm_model, text_tokenizer, max_length=200)

Generated text with seed word 'delito' and temperature 0.5:
delito, resuação de condições do artos e não deiper ader e do incnalevente destos nos relatadoriando, nos levendo, nos legatoria por invam, destos e autrato anteriodo de previdência social contribuinte, e a

Generated text with seed word 'delito' and temperature 1.0:
delito. com orique da são obrabaido cegual ou arta latomentes legação contribuinto de previdência. intigo e dalcie preão:  a a naão apavoslgo não indivíduo neu a 0

Generated text with seed word 'delito' and temperature 1.5:
delitor. pmedimo para fos áunnte.  a contribendatamento aelar do lever em nó, nãree no momocurvilo artera  dendeso. a funvoto  levendo, fecalement4 ermes pasituemal rloluual da causa, ou dima. atuepto 9rent



# Análise

Temperatura = 0.5

"juiz":

O texto gerado é previsível, refletindo termos frequentes no contexto jurídico. Palavras como "contribuinte", "previdência" e "autos" dominam. A coerência do texto é alta, mas a criatividade é baixa, resultando em uma repetição de padrões familiares.

"delito":

O resultado é consistente, mas pouco diversificado, com expressões típicas como "relatório" e "contribuinte". O texto é claro e gramaticalmente correto, porém carece de originalidade e inovação.

Temperatura = 1.0

"juiz":

Há um aumento na variedade de termos e frases, com a introdução de expressões como "condições do sistema" e "atividade previdencial". O equilíbrio entre coerência e diversidade torna o texto mais dinâmico, sem comprometer a clareza.

"delito":

O texto torna-se mais diversificado, utilizando termos mais variados como "obrigação" e "intimação". A estrutura é menos previsível e a criatividade é ampliada, mantendo a coerência adequada ao contexto jurídico.

Temperatura = 1.5

"juiz":

A criatividade é bastante elevada, resultando em frases menos claras e algumas vezes confusas, como "sestital atício" e "invalência lontorta". A diversidade de palavras é maior, mas a coesão do texto diminui significativamente.

"delito":

O texto é altamente variado, com termos inesperados como "fos áunnte" e "momocurvilo". A coerência é mais frágil, e a gramática torna-se menos consistente, com sentenças que parecem menos estruturadas e mais aleatórias.

